In [3]:
import os
import json
import torch
from tqdm import tqdm
from transformers import BartTokenizer
from transformers import BartForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

In [4]:
model = BartForSequenceClassification.from_pretrained('facebook/bart-base', num_labels=10)
model.load_state_dict(torch.load("model.pt"))


Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.out_proj.bias', 'classification_head.dense.bias', 'classification_head.out_proj.weight', 'classification_head.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [8]:
articles = []
tags = []

In [9]:
for cursor in tqdm(os.listdir("./normolized_aticles")):
    articles.append(cursor)

100%|███████████████████████████████████████████████████████████████████████| 24940/24940 [00:00<00:00, 4989789.25it/s]


In [10]:
for cursor in tqdm(articles):
    with open(f'normolized_aticles/{cursor}', encoding='utf-8') as asset:
        data = json.load(asset)
    tmp = data["tags"]
    for i in tmp:
        tags.append(i)

100%|██████████████████████████████████████████████████████████████████████████| 24940/24940 [00:04<00:00, 5870.16it/s]


In [11]:
element_counts = {}
for element in tqdm(tags):
    count = tags.count(element)
    element_counts[element] = count 

100%|█████████████████████████████████████████████████████████████████████████| 150929/150929 [03:26<00:00, 730.93it/s]


In [12]:
from operator import itemgetter

sorted_counts = sorted(element_counts.items(), key=itemgetter(1), reverse=True)

sorted_counts = sorted_counts[:10]
for element, count in sorted_counts:
    print(element, count)

проблема 676
python 607
apple 537
искусственный интеллект 507
программирование 481
игры 469
россия 430
информационная безопасность 407
машинное обучение 385
санкции 381


In [13]:
tags_top10 = []
for element, _ in sorted_counts:
        tags_top10.append(element)
        
print(tags_top10)
tags_top10_digit = []
index = 0
for tag in tags_top10:
    tmp = [tag, index]
    index += 1
    tags_top10_digit.append(tmp)

print(tags_top10_digit)

['проблема', 'python', 'apple', 'искусственный интеллект', 'программирование', 'игры', 'россия', 'информационная безопасность', 'машинное обучение', 'санкции']
[['проблема', 0], ['python', 1], ['apple', 2], ['искусственный интеллект', 3], ['программирование', 4], ['игры', 5], ['россия', 6], ['информационная безопасность', 7], ['машинное обучение', 8], ['санкции', 9]]


In [17]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

for cursor in tqdm(articles):
    with open(f'normolized_aticles/{cursor}', encoding='utf-8') as asset:
        data = json.load(asset)
    tmp = data["tags"]
    for tag in tmp:
        if tag in tags_top10:
            test_texts = data["content"]
            encoded_test_inputs = tokenizer(test_texts, padding=True, truncation=True, return_tensors='pt')
            test_input_ids = encoded_test_inputs['input_ids']
            test_attention_mask = encoded_test_inputs['attention_mask']
            
            with torch.no_grad():
                model.eval()
                test_outputs = model(input_ids=test_input_ids, attention_mask=test_attention_mask)

            test_predicted_labels = torch.argmax(test_outputs.logits, dim=1).tolist()

            for text, predicted_label in zip(test_texts, test_predicted_labels):
                print(f"Название текста: {data['article_name']}")
                print(f"Предсказанная метка: {tags_top10[predicted_label]}")
                print(f"Изначальный тег : {tag}\n")
            

  0%|                                                                                        | 0/24940 [00:00<?, ?it/s]

Название текста: Smart-lab: Nasdaq прекратит работу с резидентами из РФ с 29 апреля
Предсказанная метка: проблема
Изначальный тег : проблема



  0%|                                                                              | 5/24940 [00:06<8:33:29,  1.24s/it]

Название текста: Smart-lab: Nasdaq прекратит работу с резидентами из РФ с 29 апреля
Предсказанная метка: проблема
Изначальный тег : санкции

Название текста: Друг или враг? Стоит ли бояться искусственного интеллекта?
Предсказанная метка: искусственный интеллект
Изначальный тег : искусственный интеллект



  0%|                                                                             | 15/24940 [00:13<6:13:31,  1.11it/s]

...

Название текста: «Ведомости»: маркетплейс «AliExpress Россия» сократил до 40% персонала
Предсказанная метка: проблема
Изначальный тег : проблема



  5%|███▊                                                                       | 1250/24940 [18:28<7:18:40,  1.11s/it]

Название текста: Майские обновления безопасности для Windows вызывают сбои аутентификации AD на контроллерах доменов
Предсказанная метка: проблема
Изначальный тег : проблема

Название текста: Realme потеснил iPhone в тройке самых популярных смартфонов в РФ
Предсказанная метка: apple
Изначальный тег : россия



  5%|███▊                                                                       | 1251/24940 [18:34<5:51:40,  1.12it/s]


KeyboardInterrupt: 